# Stanford Pride Database Matching System
System to alleviate member attrition

<div>
<img src="./Images/cropped-Stanford_Pride-S@2x.png" width="300" height = "300" />
</div>

Authors: 

- Saad Saeed [Github](https://github.com/ssaeed85) | [LinkedIn](https://www.linkedin.com/in/saadsaeed85/)
- Zach Rauch [Github](https://github.com/ZachRauch) | [LinkedIn](https://www.linkedin.com/in/zach-rauch/)
- Hanis Zulmuthi [Github](https://github.com/hanis-z) | [LinkedIn](https://www.linkedin.com/in/hanis-zulmuthi/)

- Xiaohua Su [Github](https://github.com/xiaohua-su) | [LinkedIn](https://www.linkedin.com/in/xiaohua-su/)

# Overview

Nonprofit organizations want to be able to bring new members and retain them.It is vital for organizations to keep in touch with its members who are the foundation to their networks through communications about events or news. Without any method of communication, members are
no longer in touch with the organization, and its activities and are considered 'lost'. A common issue that some organizations may have is that the email provided to the organization as the main means of communication may no longer work or gets bounced once the individual graduates from said institution such as colleges, and or bootcamp. Usually, an individual might forget about updating it before they are far away. As such updating the contact method is critical to keep them in the network. Overtime, this 'lost' member issue will get larger and larger for the organization.

The purpose of this project is to help Stanford Pride address such an issue. Stanford Pride currently has ~5000 members in their database. Unfortunately, Stanford Pride has lost contact with a small portion of its member. One way Stanford Pride recognizes that it has lost contact with a member that has not chosen to opt-out of newsletter is that the newsletters was bounced. According to Stanford Pride, their members are not all using the same platform. Some chose to have subscribed to either only emails, others are only on their Facebook, LinkedIn group and a small minority
interacts with Stanford using multiple platform. As such, Stanford Pride hopes to be able to rectify the issue of lost members by
updating the individual's contact information in order to bring/keep them in the network once again.

Our goal for this project is to help Stanford Pride be able to update this information in a more efficient way. We improved the efficiency by using a cosine similar model to provide a list of individuals from the Stanford Pride database with the individual from their Mailchimp database. This way, the chair in-charge of updating their database does not need to look up multiple potential people on their Stanford Database before deciding if they are the same individual. They now have a list of potential matches with information about them to compare against.

From Stanford Pride:
> A nonprofit organization, such as Stanford Pride, strives by attracting and retaining members. 
> It is vital for the organization to stay in touch with its members. 
> The main means to achieve this is the sending of newsletters via e-mail. 
> Members are not likely to keep informed of the organization’s activity on their own. We only stay in their minds by regularly pushing news out to them.
Members do not always subscribe to other sources of information about the organization’s activities. 
> For example, Stanford Pride has approximately 4,400 members in its database, out of which about 3,700 currently have valid e-mail addresses. 
> Only 1,600 are part of our Facebook group, and 400 in our LinkedIn group. 
> Therefore, our monthly e-mail newsletter is our sole means to reach about 2,100 members – almost half of our total membership.


# Methodology

In order to be able to tackle this issue, we received a database of their mailchimp and Stanford Pride.
Due to the sensitive information in the data, we only received a csv file and an Excel file that contained
the raw data's column names. From there, a fake dataset was created for them. In the datasets, we included some potential issues discussed as well as add some potential issues that we believe may appear down the line.How the fake dataset was created can be found [here](https://github.com/ssaeed85/Equalithon-team6/blob/main/FakeDataset.ipynb). Only the columns that had a match in the Mailchimp dataset were filled.

Using the fake datasets, we decided to use cosine similarity to create a list of individuals that matched on the first name. A score of how similar all the records in the Stanford Pride database is produced out of the cosine model that will then be used to sort the list.

## Imports

In [1]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

import regex as re
import pandas as pd
import numpy as np
import random 
import pycountry
np.random.seed = 42
random.seed(42)

pd.set_option('display.max_columns', None)
pd.set_option('display.min_rows', 10)

In [2]:
mailchimp_data = pd.read_csv('./data/MailChimp cleaned records headers.csv')
pd.set_option('display.max_columns', None)
pd.set_option('display.min_rows', 10)
mailchimp_data

,Email Address,First Name,Last Name,Board Member,Gender,Chapter,Reunion Year,Country,Degree,MEMBER_RATING,OPTIN_TIME,OPTIN_IP,CONFIRM_TIME,CONFIRM_IP,LATITUDE,LONGITUDE,GMTOFF,DSTOFF,TIMEZONE,CC,REGION,CLEAN_TIME,CLEAN_CAMPAIGN_TITLE,CLEAN_CAMPAIGN_ID,LEID,EUID,NOTES,TAGS


The mailchimp dataset is derived from the salesforce database. This Cleaned dataset would only include emails that have bounced for one reason or another. It will not include any members who have opted out of the email service or those that are receiving emails without any complications. It is important to note that updating information within salesforce for those who have opted out may re-instantiate the email services, so we are specifically working with cleaned/bounced records. Additionally, the datasets created for those who are Subscribed and Unsubscribed will have different column names than the Cleaned dataset here (ie Clean_Time, Clean_campaign_title, ect). Therefore, any function created here may not work for those datasets.

In [3]:
saa_pride_data = pd.read_excel('./data/SAA Pride member reports headings.xlsx')
saa_pride_data.reset_index(inplace = True)
saa_pride_data

,index,pref_mail_name,pref_class_year,home_city,home_state_code,home_country,home_phone_area_code,home_phone_number,home_email_address,bus_city,bus_state_code,bus_country,bus_phone_area_code,bus_phone_number,bus_email_address,first_name,last_name,pref_name_sort,email_switch,saa_email_address,gsb_email_address,other_email_address,pref_phone_area_code,pref_phone_number,pref_phone_addr_type,memb_status_desc,short_degree_string,parent_degree_string,short_degree_string_spouse,parent_degree_string_spouse,primary_sort_name,plan_name,primary_ind


The Stanford Alumni Association has it's own dataset that may or may not have additional or more recent data on some members. It may also have outdated data. Students are given an email but when they become alumni the email needs to be updated. Whether it is updated to specifically an 'alumni.stanford.edu' address or to another would be at the students discrepancy and isn't always updated.

## Helper Functions

In [4]:
def removeEmailDomain(email):
    '''
    removes the email portion of an email.
    Ex. xxxx@gmail.com becomes xxxx
    '''
    email = str(email)
    #if email string contains @ symbol return the handle
    if email.find('@')>-1:
        return email.split('@')[0]
    else:
        return 'n/a'

In [5]:
def mapCountry(countryStr):
    '''
    Maps a Country
    '''
    if countryStr not in [np.nan, None, 'nan','*', '','N/A','n/a']:
        return pycountry.countries.search_fuzzy(countryStr)[0].alpha_3
    return 'n/a'

In [6]:
def ohe(df, column): 
    '''
    Converts the strings in a column into their own columns
    '''
    for col in column:
        train = df[[col]]
        ohe = OneHotEncoder(sparse=False, handle_unknown="error")
        ohe.fit(train)
        encoded_train = ohe.transform(train)
        col_names = [f"{col}_{f}" for f in ohe.get_feature_names()]
        encoded_train = pd.DataFrame(encoded_train,
                                     columns=col_names, index=df.index)
        df = pd.concat([df, encoded_train], axis=1)
        
    return df

In [7]:
def parseDegreeCol(df,deg_colName='short_degree_string'):
    '''
    OHE the degrees in the fake Stanford Database
    '''
    vectorizer = CountVectorizer()
    
    t = vectorizer.fit_transform(df[deg_colName])
    
    # Remove original degree column from original dataframe
    df = df.drop(columns=[deg_colName])
    
    # Create new vectorized degree columns and concatenate
    deg_df =  pd.DataFrame(t.toarray(),columns=vectorizer.get_feature_names(),index = df.index)
    if 'nan' in deg_df.columns:
        deg_df.drop(columns = 'nan')
    return pd.concat([df,deg_df],axis = 1).fillna(0)

In [8]:
def cleanDegreeStr(degree):
    '''
    Clean the degree strings to allow for better handling
    '''
    no_nums_puncs = re.sub('[;,\'*0-9]', ' ', degree).strip()
    return re.sub('[ ]+', ' ', no_nums_puncs).strip()

### Pokemon Data

In [9]:
# read in our fake Stanford dataset
df_saa_original = pd.read_excel('./data/SAA_Pokemon_FakeDB.xlsx')
df_saa_original

,pref_mail_name,pref_class_year,home_city,home_state_code,home_country,home_phone_area_code,home_phone_number,home_email_address,bus_city,bus_state_code,bus_country,bus_phone_area_code,bus_phone_number,bus_email_address,first_name,last_name,pref_name_sort,email_switch,saa_email_address,gsb_email_address,other_email_address,pref_phone_area_code,pref_phone_number,pref_phone_addr_type,memb_status_desc,short_degree_string,parent_degree_string,short_degree_string_spouse,parent_degree_string_spouse,primary_sort_name,plan_name,primary_ind
0,NaN,2004.0,Shanghai,NaN,*,NaN,*,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Growlithe,Ice,NaN,NaN,*,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,Madrid,NaN,China,NaN,NaN,weedleg4046@stanfordalumni.org,NaN,NaN,China,NaN,NaN,NaN,Weedle,Grass,NaN,NaN,w.grass5053@alumni.stanford.edu,NaN,NaN,NaN,NaN,NaN,NaN,'82,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,Seoul,NaN,Kuwait,NaN,NaN,aerodactyl.electric2974@alumni.stanford.edu,NaN,NaN,Kuwait,NaN,NaN,NaN,Aerodactyl,Electric,NaN,NaN,*,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,London,NaN,NaN,NaN,*,*,NaN,NaN,Japan,NaN,NaN,NaN,Pinsir,Fire,NaN,pinsirfire4582@gmail.com,*,NaN,NaN,NaN,NaN,NaN,NaN,"JD '94, PhD '97",NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,London,NaN,USA,NaN,775 0678-214,*,NaN,NaN,NaN,NaN,NaN,NaN,Horsea,Ice,NaN,hice7313@stanfordalumni.org,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,NaN,NaN,Boston,MA,Japan,NaN,NaN,*,NaN,NaN,NaN,NaN,NaN,NaN,Smeargle,Electric,NaN,smeargleelectric9444@gmail.com,*,NaN,NaN,NaN,NaN,NaN,NaN,"PhD '96, MBA '87",NaN,NaN,NaN,NaN,NaN,NaN
3996,NaN,2000.0,Shanghai,NaN,NaN,NaN,*,kabutops.steel1285@stanfordalumni.org,NaN,NaN,China,NaN,NaN,NaN,Kabutops,Steel,NaN,k.steel5317@alumni.stanford.edu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BS '93,NaN,NaN,NaN,NaN,NaN,NaN
3997,NaN,NaN,NaN,NaN,Kuwait,NaN,NaN,*,NaN,NaN,NaN,NaN,NaN,NaN,Slowking,Dragon,NaN,NaN,slowkingd5563@stanfordalumni.org,NaN,NaN,NaN,NaN,NaN,NaN,"PhD '84, JD '93",NaN,NaN,NaN,NaN,NaN,NaN
3998,NaN,NaN,NaN,NaN,Japan,NaN,NaN,larvitar.electric9778@stanfordalumni.org,NaN,NaN,United States,NaN,NaN,NaN,Larvitar,Electric,NaN,l.electric7920@stanfordalumni.org,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MBA '92, '98, JD '86",NaN,NaN,NaN,NaN,NaN,NaN


# Preprocess SAA dataframe

Since the SAA dataset contain more information than the Mailchimp, we will only select the relevant information (columns). Not only that, but by limiting it to just relevant information, when we have to convert the Mailchimp point to have the same information as those in Stanford dataset; fewer nulls will be filled in. In addition, this will help with the speed of any process that will be performed later on as well since there is less information.

In [10]:
# Filter necessary columns
df_saa = df_saa_original.filter(['first_name', 'last_name',
                        'home_country', 'home_email_address', 'short_degree_string',
                        'bus_email_address', 'bus_country', 'email_switch',
                        'saa_email_address', 'gsb_email_address', 'other_email_address'])
df_saa

,first_name,last_name,home_country,home_email_address,short_degree_string,bus_email_address,bus_country,email_switch,saa_email_address,gsb_email_address,other_email_address
0,Growlithe,Ice,*,NaN,NaN,NaN,NaN,NaN,*,NaN,NaN
1,Weedle,Grass,China,weedleg4046@stanfordalumni.org,'82,NaN,China,NaN,w.grass5053@alumni.stanford.edu,NaN,NaN
2,Aerodactyl,Electric,Kuwait,aerodactyl.electric2974@alumni.stanford.edu,NaN,NaN,Kuwait,NaN,*,NaN,NaN
3,Pinsir,Fire,NaN,*,"JD '94, PhD '97",NaN,Japan,pinsirfire4582@gmail.com,*,NaN,NaN
4,Horsea,Ice,USA,*,NaN,NaN,NaN,hice7313@stanfordalumni.org,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
3995,Smeargle,Electric,Japan,*,"PhD '96, MBA '87",NaN,NaN,smeargleelectric9444@gmail.com,*,NaN,NaN
3996,Kabutops,Steel,NaN,kabutops.steel1285@stanfordalumni.org,BS '93,NaN,China,k.steel5317@alumni.stanford.edu,NaN,NaN,NaN
3997,Slowking,Dragon,Kuwait,*,"PhD '84, JD '93",NaN,NaN,NaN,slowkingd5563@stanfordalumni.org,NaN,NaN
3998,Larvitar,Electric,Japan,larvitar.electric9778@stanfordalumni.org,"MBA '92, '98, JD '86",NaN,United States,l.electric7920@stanfordalumni.org,NaN,NaN,NaN


In [11]:
# replace nulls
df_saa.fillna('n/a',inplace=True)
df_saa

,first_name,last_name,home_country,home_email_address,short_degree_string,bus_email_address,bus_country,email_switch,saa_email_address,gsb_email_address,other_email_address
0,Growlithe,Ice,*,n/a,n/a,n/a,n/a,n/a,*,n/a,n/a
1,Weedle,Grass,China,weedleg4046@stanfordalumni.org,'82,n/a,China,n/a,w.grass5053@alumni.stanford.edu,n/a,n/a
2,Aerodactyl,Electric,Kuwait,aerodactyl.electric2974@alumni.stanford.edu,n/a,n/a,Kuwait,n/a,*,n/a,n/a
3,Pinsir,Fire,n/a,*,"JD '94, PhD '97",n/a,Japan,pinsirfire4582@gmail.com,*,n/a,n/a
4,Horsea,Ice,USA,*,n/a,n/a,n/a,hice7313@stanfordalumni.org,n/a,n/a,n/a
...,...,...,...,...,...,...,...,...,...,...,...
3995,Smeargle,Electric,Japan,*,"PhD '96, MBA '87",n/a,n/a,smeargleelectric9444@gmail.com,*,n/a,n/a
3996,Kabutops,Steel,n/a,kabutops.steel1285@stanfordalumni.org,BS '93,n/a,China,k.steel5317@alumni.stanford.edu,n/a,n/a,n/a
3997,Slowking,Dragon,Kuwait,*,"PhD '84, JD '93",n/a,n/a,n/a,slowkingd5563@stanfordalumni.org,n/a,n/a
3998,Larvitar,Electric,Japan,larvitar.electric9778@stanfordalumni.org,"MBA '92, '98, JD '86",n/a,United States,l.electric7920@stanfordalumni.org,n/a,n/a,n/a


In [12]:
#clean column names for easier handling
for col in df_saa.columns:    
    df_saa[col] = df_saa[col].astype(str).str.lower().str.strip()
df_saa

,first_name,last_name,home_country,home_email_address,short_degree_string,bus_email_address,bus_country,email_switch,saa_email_address,gsb_email_address,other_email_address
0,growlithe,ice,*,n/a,n/a,n/a,n/a,n/a,*,n/a,n/a
1,weedle,grass,china,weedleg4046@stanfordalumni.org,'82,n/a,china,n/a,w.grass5053@alumni.stanford.edu,n/a,n/a
2,aerodactyl,electric,kuwait,aerodactyl.electric2974@alumni.stanford.edu,n/a,n/a,kuwait,n/a,*,n/a,n/a
3,pinsir,fire,n/a,*,"jd '94, phd '97",n/a,japan,pinsirfire4582@gmail.com,*,n/a,n/a
4,horsea,ice,usa,*,n/a,n/a,n/a,hice7313@stanfordalumni.org,n/a,n/a,n/a
...,...,...,...,...,...,...,...,...,...,...,...
3995,smeargle,electric,japan,*,"phd '96, mba '87",n/a,n/a,smeargleelectric9444@gmail.com,*,n/a,n/a
3996,kabutops,steel,n/a,kabutops.steel1285@stanfordalumni.org,bs '93,n/a,china,k.steel5317@alumni.stanford.edu,n/a,n/a,n/a
3997,slowking,dragon,kuwait,*,"phd '84, jd '93",n/a,n/a,n/a,slowkingd5563@stanfordalumni.org,n/a,n/a
3998,larvitar,electric,japan,larvitar.electric9778@stanfordalumni.org,"mba '92, '98, jd '86",n/a,united states,l.electric7920@stanfordalumni.org,n/a,n/a,n/a


In [13]:
#remove email domains using removeEmailDomain helper function
email_cols = ['home_email_address', 'bus_email_address', 'email_switch',
              'saa_email_address', 'gsb_email_address', 'other_email_address']
for email in email_cols:
    df_saa[email] = df_saa[email].apply(removeEmailDomain)
df_saa

,first_name,last_name,home_country,home_email_address,short_degree_string,bus_email_address,bus_country,email_switch,saa_email_address,gsb_email_address,other_email_address
0,growlithe,ice,*,n/a,n/a,n/a,n/a,n/a,n/a,n/a,n/a
1,weedle,grass,china,weedleg4046,'82,n/a,china,n/a,w.grass5053,n/a,n/a
2,aerodactyl,electric,kuwait,aerodactyl.electric2974,n/a,n/a,kuwait,n/a,n/a,n/a,n/a
3,pinsir,fire,n/a,n/a,"jd '94, phd '97",n/a,japan,pinsirfire4582,n/a,n/a,n/a
4,horsea,ice,usa,n/a,n/a,n/a,n/a,hice7313,n/a,n/a,n/a
...,...,...,...,...,...,...,...,...,...,...,...
3995,smeargle,electric,japan,n/a,"phd '96, mba '87",n/a,n/a,smeargleelectric9444,n/a,n/a,n/a
3996,kabutops,steel,n/a,kabutops.steel1285,bs '93,n/a,china,k.steel5317,n/a,n/a,n/a
3997,slowking,dragon,kuwait,n/a,"phd '84, jd '93",n/a,n/a,n/a,slowkingd5563,n/a,n/a
3998,larvitar,electric,japan,larvitar.electric9778,"mba '92, '98, jd '86",n/a,united states,l.electric7920,n/a,n/a,n/a


In [14]:
# Cleaning degree string column using cleanDegreeStr helper function
df_saa.short_degree_string = df_saa.short_degree_string.apply(cleanDegreeStr)
df_saa

,first_name,last_name,home_country,home_email_address,short_degree_string,bus_email_address,bus_country,email_switch,saa_email_address,gsb_email_address,other_email_address
0,growlithe,ice,*,n/a,n/a,n/a,n/a,n/a,n/a,n/a,n/a
1,weedle,grass,china,weedleg4046,,n/a,china,n/a,w.grass5053,n/a,n/a
2,aerodactyl,electric,kuwait,aerodactyl.electric2974,n/a,n/a,kuwait,n/a,n/a,n/a,n/a
3,pinsir,fire,n/a,n/a,jd phd,n/a,japan,pinsirfire4582,n/a,n/a,n/a
4,horsea,ice,usa,n/a,n/a,n/a,n/a,hice7313,n/a,n/a,n/a
...,...,...,...,...,...,...,...,...,...,...,...
3995,smeargle,electric,japan,n/a,phd mba,n/a,n/a,smeargleelectric9444,n/a,n/a,n/a
3996,kabutops,steel,n/a,kabutops.steel1285,bs,n/a,china,k.steel5317,n/a,n/a,n/a
3997,slowking,dragon,kuwait,n/a,phd jd,n/a,n/a,n/a,slowkingd5563,n/a,n/a
3998,larvitar,electric,japan,larvitar.electric9778,mba jd,n/a,united states,l.electric7920,n/a,n/a,n/a


# Preprocess Mailchimp dataframe

In [15]:
df_mailchimp_original = pd.read_csv('./data/Fake_MailChimp_cleaned_Pokemon.csv')
df_mailchimp_original

,Email Address,First Name,Last Name,Board Member,Gender,Chapter,Reunion Year,Country,Degree,MEMBER_RATING,OPTIN_TIME,OPTIN_IP,CONFIRM_TIME,CONFIRM_IP,LATITUDE,LONGITUDE,GMTOFF,DSTOFF,TIMEZONE,CC,REGION,CLEAN_TIME,CLEAN_CAMPAIGN_TITLE,CLEAN_CAMPAIGN_ID,LEID,EUID,NOTES,TAGS
0,slakoth.normal3945@gmail.com,Slakoth,Normal,False,NaN,Texas,NaN,USA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,e.rock7454@gmail.com,Espeon,Rock,True,F,DC Area,NaN,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,rhydonghost7966@alumni.stanford.edu,Rhydon,Ghost,False,M,Bay Area,NaN,USA,MBA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,porygong9247@stanfordalumni.org,Porygon,Grass,False,M,Bay Area,NaN,Japan,MS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,tangelagrass1376@gmail.com,Tangela,Grass,False,NaN,New England,NaN,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,c.electric7518@gmail.com,Chansey,Steel,True,F,Other US,NaN,USA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,blissey.ghost4154@gmail.com,Blissey,Ghost,False,M,New England,NaN,Macao Special Administrative Region of China,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


The mailchimp dataset contains information that is not found in the SAA as such it is not helpful to keep them in our dataframe and unnecessary since we know that the dataset contains ONLY BOUNCED individuals. 

In [16]:
# Filter necessary columns
df_mailchimp = df_mailchimp_original.filter(['First Name', 'Last Name', 'Email Address',
                                         'Degree', 'Country'])
df_mailchimp

,First Name,Last Name,Email Address,Degree,Country
0,Slakoth,Normal,slakoth.normal3945@gmail.com,NaN,USA
1,Espeon,Rock,e.rock7454@gmail.com,NaN,United States
2,Rhydon,Ghost,rhydonghost7966@alumni.stanford.edu,MBA,USA
3,Porygon,Grass,porygong9247@stanfordalumni.org,MS,Japan
4,Tangela,Grass,tangelagrass1376@gmail.com,NaN,United States
5,Chansey,Steel,c.electric7518@gmail.com,NaN,USA
6,Blissey,Ghost,blissey.ghost4154@gmail.com,NaN,Macao Special Administrative Region of China


In [17]:
# Replace nulls with 'n/a'
df_mailchimp.fillna('n/a',inplace=True)
df_mailchimp

,First Name,Last Name,Email Address,Degree,Country
0,Slakoth,Normal,slakoth.normal3945@gmail.com,n/a,USA
1,Espeon,Rock,e.rock7454@gmail.com,n/a,United States
2,Rhydon,Ghost,rhydonghost7966@alumni.stanford.edu,MBA,USA
3,Porygon,Grass,porygong9247@stanfordalumni.org,MS,Japan
4,Tangela,Grass,tangelagrass1376@gmail.com,n/a,United States
5,Chansey,Steel,c.electric7518@gmail.com,n/a,USA
6,Blissey,Ghost,blissey.ghost4154@gmail.com,n/a,Macao Special Administrative Region of China


In [18]:
#clean column names for easier handling
for col in ['First Name','Last Name','Email Address','Degree']:
    df_mailchimp[col] = df_mailchimp[col].str.lower().str.strip()
df_mailchimp

,First Name,Last Name,Email Address,Degree,Country
0,slakoth,normal,slakoth.normal3945@gmail.com,n/a,USA
1,espeon,rock,e.rock7454@gmail.com,n/a,United States
2,rhydon,ghost,rhydonghost7966@alumni.stanford.edu,mba,USA
3,porygon,grass,porygong9247@stanfordalumni.org,ms,Japan
4,tangela,grass,tangelagrass1376@gmail.com,n/a,United States
5,chansey,steel,c.electric7518@gmail.com,n/a,USA
6,blissey,ghost,blissey.ghost4154@gmail.com,n/a,Macao Special Administrative Region of China


In [19]:
#remove email domains using removeEmailDomain helper function
df_mailchimp['Email Address'] = df_mailchimp['Email Address'].apply(removeEmailDomain)
df_mailchimp

,First Name,Last Name,Email Address,Degree,Country
0,slakoth,normal,slakoth.normal3945,n/a,USA
1,espeon,rock,e.rock7454,n/a,United States
2,rhydon,ghost,rhydonghost7966,mba,USA
3,porygon,grass,porygong9247,ms,Japan
4,tangela,grass,tangelagrass1376,n/a,United States
5,chansey,steel,c.electric7518,n/a,USA
6,blissey,ghost,blissey.ghost4154,n/a,Macao Special Administrative Region of China


In [20]:
# Convert Country to 3 digit code
# df_mailchimp.Country = df_mailchimp.Country.apply(mapCountry)
# df_mailchimp

Converting to 3-char country code is time consuming. Reserving that conversion after subsetting.

In [21]:
# Cleaning degree string column using cleanDegreeStr helper function
df_mailchimp.Degree = df_mailchimp.Degree.apply(cleanDegreeStr)
df_mailchimp

,First Name,Last Name,Email Address,Degree,Country
0,slakoth,normal,slakoth.normal3945,n/a,USA
1,espeon,rock,e.rock7454,n/a,United States
2,rhydon,ghost,rhydonghost7966,mba,USA
3,porygon,grass,porygong9247,ms,Japan
4,tangela,grass,tangelagrass1376,n/a,United States
5,chansey,steel,c.electric7518,n/a,USA
6,blissey,ghost,blissey.ghost4154,n/a,Macao Special Administrative Region of China


# Find Best Match for each MailChimp record

Finding the best match for each mail chimp record is a 3 step process:
- Map the MailChimp record to the Stanford Alumni dataframe
- Subset the entire dataframe by the first name*
- Calculate the cosine similarity for the resulting subset

Few edge cases to note:
- Subset by just the first name, in case a user has changed their last name but change their emails or contact info
- We check for similarity of the 'bounced' MailChimp email handle against *all* possible email fields
- We check for similarity of the 'bounced' MailChimp country field against *all* possible country fields

*last names can change due to marriage as such we decided to only focus on first name. We acknowledge that individuals can also change their first name as well, but we came to the conclusion that it would only affect a small portion of individuals in the database. Should this statistic change in the future then new considerations on how to filter should be discussed.

In [22]:
def map_MailChimpData_to_SAA_DF(mc_rec_idx,df_mc=df_mailchimp):
    '''
    Creates a SAA df for the mailchimp individual. Because SAA has multiple
    fields for email addresses and mailchimp only has one. The one email is then
    filled in for all the different email addresses.
    
    '''
    mc_rec = df_mc.iloc[mc_rec_idx]
    target_dict = {'first_name': mc_rec['First Name'], 
                   'last_name': mc_rec['Last Name'],
                   
                   'home_email_address': mc_rec['Email Address'],
                   'bus_email_address': mc_rec['Email Address'],
                   'email_switch': mc_rec['Email Address'],
                   'saa_email_address': mc_rec['Email Address'],
                   'gsb_email_address': mc_rec['Email Address'],
                   'other_email_address': mc_rec['Email Address'],
                   
                    'home_country': mc_rec['Country'],                   
                    'bus_country': mc_rec['Country'],          
                   
                    'short_degree_string': mc_rec['Degree']
                  
                  }
    df =  pd.DataFrame(target_dict,index=['mc_'+str(mc_rec_idx)])
    return df

In [23]:
# verifying our function worked as desired
map_MailChimpData_to_SAA_DF(0).iloc[0]

first_name                        slakoth
last_name                          normal
home_email_address     slakoth.normal3945
bus_email_address      slakoth.normal3945
email_switch           slakoth.normal3945
saa_email_address      slakoth.normal3945
gsb_email_address      slakoth.normal3945
other_email_address    slakoth.normal3945
home_country                          USA
bus_country                           USA
short_degree_string                   n/a
Name: mc_0, dtype: object

In [24]:
print(df_mailchimp.shape)

(7, 5)


## Expected Result

The result returned is a dictionary of
- Dataframe of data in order of score (key: `df`)
- List of index of records in stanford alumni dataframe in order of score (key: `idx`)
- List of cosin similarity scores (key: `score`)

We are making it into a dictionary in order for us to be able to access the different aspects of the data such as the score,dataframe etc.. Not only that but potentially this may be helpful to UI/UX designers.

In [34]:
results_dict = {}

for i in range(0,df_mailchimp.shape[0]):
    user_dict={}
    #map the data of a mailchimp record to the format in SAA df
    df_mapped_mc_rec = map_MailChimpData_to_SAA_DF(i)
    
    #subsetting the original SAA df by the first name
    df_saa_subset = df_saa[df_saa['first_name'] == df_mapped_mc_rec.iloc[0]['first_name']]
    
    #Concattenating the 2 dataframes together
    df_mc_and_saa_subset = pd.concat([df_mapped_mc_rec,df_saa_subset], axis = 0)
    
    #Changing degree column to vectorized columns
    df_mc_and_saa_subset = parseDegreeCol(df=df_mc_and_saa_subset,deg_colName='short_degree_string')
    
    #Map the country to their 3 digit codes
    df_mc_and_saa_subset.home_country = df_mc_and_saa_subset.home_country.apply(mapCountry)
    df_mc_and_saa_subset.bus_country = df_mc_and_saa_subset.bus_country.apply(mapCountry)

    #one hot encoding our dataframe
    ohe_df = ohe(df_mc_and_saa_subset, df_mc_and_saa_subset.columns)
    #dropping off the columns that have been ohe since they are still present
    ohe_df.drop(columns = df_mc_and_saa_subset.columns, inplace = True)
    
    # setting our y for cosine similarity
    y = np.array(ohe_df.iloc[0])
    y = y.reshape(1,-1)
    
    # using Cosine Similarity
    cos_sim = cosine_similarity(ohe_df, y)
    
    #converting the cosine score array into a df
    cos_sim = pd.DataFrame(data=cos_sim, index=ohe_df.index).sort_values(by=0, ascending=False) #[1:]
    
    #saving the cos_sim index which should be the SAA indexes
    results = list(cos_sim.index)
    
    #locating these indexes in our subset
    results_df = df_mc_and_saa_subset.loc[results]

    # save the cosine df as a value to the cosine_sim_result
    user_dict['cosine_sim_result'] = results_df
    
    #original data that can be found the SAA database that matches with the MailChimp
    user_dict['SAA_query_result'] = df_saa_original.iloc[results[1:]]
    
    # creates a dictonary 
    results_dict[i] = user_dict

In [35]:
#inspecting user_dict
user_dict

{'cosine_sim_result':      first_name last_name home_email_address  bus_email_address  \
 mc_6    blissey     ghost  blissey.ghost4154  blissey.ghost4154   
 600     blissey     ghost                n/a                n/a   
 754     blissey      rock                n/a                n/a   
 802     blissey     steel                n/a                n/a   
 1600    blissey    dragon                n/a                n/a   
 390     blissey     fairy         bfairy6293                n/a   
 850     blissey       ice                n/a                n/a   
 977     blissey       ice                n/a                n/a   
 1430    blissey      rock                n/a                n/a   
 1648    blissey      fire        blisseyf465                n/a   
 2402    blissey       ice           bice5800                n/a   
 858     blissey      rock                n/a   blissey.rock4438   
 1545    blissey      dark       blisseyd5981                n/a   
 1935    blissey     water 

In [37]:
#testing the results_dict
#need index of the mailchimp record you want the score for before you can access the table
results_dict[0]['cosine_sim_result']

,first_name,last_name,home_email_address,bus_email_address,email_switch,saa_email_address,gsb_email_address,other_email_address,home_country,bus_country,ba,bs,jd,ma,mba,md,ms,phd
mc_0,slakoth,normal,slakoth.normal3945,slakoth.normal3945,slakoth.normal3945,slakoth.normal3945,slakoth.normal3945,slakoth.normal3945,USA,USA,0,0,0,0,0,0,0,0
1290,slakoth,normal,n/a,n/a,s.normal5961,n/a,n/a,n/a,USA,n/a,0,0,0,0,0,0,0,0
1967,slakoth,ghost,slakoth.ghost5782,n/a,slakothg316,sghost4008,n/a,n/a,USA,n/a,0,0,0,0,0,0,0,0
74,slakoth,normal,slakoth.normal3945,n/a,n/a,n/a,n/a,n/a,USA,JPN,1,0,0,0,1,0,0,1
2052,slakoth,water,slakothw1017,n/a,slakothw808,n/a,n/a,n/a,KWT,USA,0,0,0,1,0,0,0,0
2353,slakoth,poison,n/a,s.poison6761,n/a,n/a,n/a,n/a,n/a,USA,0,0,0,0,1,0,0,0
503,slakoth,rock,n/a,n/a,srock5364,n/a,n/a,n/a,n/a,CHN,0,1,0,0,0,0,0,0
776,slakoth,ground,n/a,n/a,n/a,n/a,n/a,n/a,n/a,n/a,1,0,0,0,0,0,0,0
1095,slakoth,fighting,n/a,n/a,slakothfighting1488,s.fighting883,n/a,n/a,USA,n/a,0,0,1,0,0,1,0,0
1485,slakoth,ground,sground648,sground2975,sground5309,slakothground1530,n/a,n/a,n/a,USA,0,0,0,0,0,1,1,0


In [38]:
#testing the results_dict
results_dict[0]['SAA_query_result']

,pref_mail_name,pref_class_year,home_city,home_state_code,home_country,home_phone_area_code,home_phone_number,home_email_address,bus_city,bus_state_code,bus_country,bus_phone_area_code,bus_phone_number,bus_email_address,first_name,last_name,pref_name_sort,email_switch,saa_email_address,gsb_email_address,other_email_address,pref_phone_area_code,pref_phone_number,pref_phone_addr_type,memb_status_desc,short_degree_string,parent_degree_string,short_degree_string_spouse,parent_degree_string_spouse,primary_sort_name,plan_name,primary_ind
1290,NaN,NaN,Seoul,NaN,USA,NaN,735 1117-916,*,NaN,NaN,NaN,NaN,NaN,NaN,Slakoth,Normal,NaN,s.normal5961@alumni.stanford.edu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1967,NaN,NaN,London,NaN,United States,NaN,NaN,slakoth.ghost5782@stanfordalumni.org,NaN,NaN,NaN,NaN,NaN,NaN,Slakoth,Ghost,NaN,slakothg316@alumni.stanford.edu,sghost4008@stanfordalumni.org,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74,NaN,NaN,Seoul,NaN,USA,NaN,*,slakoth.normal3945@alumni.stanford.edu,NaN,NaN,Japan,NaN,NaN,NaN,Slakoth,Normal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"BA '99, PhD '85, MBA '89",NaN,NaN,NaN,NaN,NaN,NaN
2052,NaN,NaN,*,*,Kuwait,NaN,339 5237-035,slakothw1017@gmail.com,NaN,NaN,USA,NaN,NaN,NaN,Slakoth,Water,NaN,slakothw808@stanfordalumni.org,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"'98, MA '94",NaN,NaN,NaN,NaN,NaN,NaN
2353,NaN,2001.0,NaN,NaN,NaN,NaN,*,NaN,NaN,NaN,USA,NaN,NaN,s.poison6761@stanfordalumni.org,Slakoth,Poison,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MBA '91,NaN,NaN,NaN,NaN,NaN,NaN
503,NaN,2018.0,Beijing,NaN,NaN,NaN,*,*,NaN,NaN,China,NaN,NaN,NaN,Slakoth,Rock,NaN,srock5364@stanfordalumni.org,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BS '82,NaN,NaN,NaN,NaN,NaN,NaN
776,NaN,NaN,Seoul,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Slakoth,Ground,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BA '90,NaN,NaN,NaN,NaN,NaN,NaN
1095,NaN,NaN,London,NaN,USA,NaN,NaN,*,NaN,NaN,NaN,NaN,NaN,NaN,Slakoth,Fighting,NaN,slakothfighting1488@gmail.com,s.fighting883@alumni.stanford.edu,NaN,NaN,NaN,NaN,NaN,NaN,"MD '99, '80, JD '88",NaN,NaN,NaN,NaN,NaN,NaN
1485,NaN,NaN,Shanghai,NaN,NaN,NaN,NaN,sground648@stanfordalumni.org,NaN,NaN,USA,NaN,NaN,sground2975@stanfordalumni.org,Slakoth,Ground,NaN,sground5309@stanfordalumni.org,slakothground1530@stanfordalumni.org,NaN,NaN,NaN,NaN,NaN,NaN,"MD '80, MS '93",NaN,NaN,NaN,NaN,NaN,NaN
2709,NaN,NaN,NaN,NaN,NaN,NaN,NaN,s.dragon440@alumni.stanford.edu,NaN,NaN,United States,NaN,NaN,NaN,Slakoth,Dragon,NaN,NaN,sdragon3285@stanfordalumni.org,NaN,NaN,NaN,NaN,NaN,NaN,"MA '93, MBA '87, MA '98",NaN,NaN,NaN,NaN,NaN,NaN


# App

- app made to be used locally, not deploying it on the streamlit.
- What shows up first is the Mailchimp dataset

# Next Steps

- The model can be further optimized and tuned to account for the rare issues that arises from the datasets.


- Use the Salesforce information through its API. This filtering and model only used the dataset from Mailchimp which is in sync with the Salesforce database. Unfortunately, the Mailchimp data has very limited information compared to its mother database Salesforce as such the information used to match is very limited. As such, being able to potentially match the individual on the bounced Mailchimp data to the Salesforce to increase the potential areas of matches will likely yield better matches to the Stanford Database.


- Further investigation of Mailchimp and Salesforce platform. Due to time restraint, we were unable to look into how both of these platforms integrate with each other and work. We would want to investigate if there is a way for us do an API call from Salesforce to Mailchimp. This way, we can potentially get a list of bounced individuals from Mailchimp in Salesforce and implement our model there as there is developer console available on Salesforce.


- Take a proactive approach. We would suggest using the salesforce dataset to check for recent grads or to-be-graduates to reach out before they lose their student emails and ask for updated contact information and their subsequent plans after graduation. It would be easier to update records proactively when we still have accurate contact information. For subsequent plans after graduation, this would be to keeping location information of members to more accurately send regional events and functions. 

# Additional Notes

- Mailchimp gets live data from Salesforce each time it produces a report. As such, an individual's information on Mailchimp reflects that of Salesforce. Unfortunately, information that Mailchimp produces does not get sent to the Salesforce database as such within Salesforce, there is no saved information on whether an email has been bounced on MailChimp. As such, we'd have to use a more local method to get additional information on an individual from the Salesforce. Unless if it is possible to create a specialized report on Mailchimp which contained most of the Salesforce information. But that would need to be investigated further.


- We recently found that Salesforce and Mailchimp each have a REST API that can be used to pull information. However, further discussion will need to be had about these APIs since Stanford Pride is using a Non-Profit Version of Salesforce. As such, limitations might be apply by the platforms. Both of these APIs would need heavy investigation on what their limitations would be for Stanford Pride.
    - [SalesForce API Documentation](https://developer.salesforce.com/docs/atlas.en-us.api_rest.meta/api_rest/intro_what_is_rest_api.htm)
    - [MailChimp API Documentation](https://mailchimp.com/developer/)